In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
# Check cuda version
!nvcc --version

# Install required packages
%tensorflow_version 2.x
!pip install gast==0.2.2
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0

In [ ]:
%cd gdrive/MyDrive/Mini-Project/snip

In [ ]:
# Define a dummy class
class Args:
    def __init__(self):
        self.arch = 'lenet300'
        self.aug_kinds = ['fliplr', 'translate_px']
        self.batch_size = 500
        self.check_interval = 100
        self.datasource = 'mnist'
        self.decay_boundaries = [5000, 10000, 15000, 20000, 25000]
        self.decay_values = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
        self.initializer_b_ap = 'zeros'
        self.initializer_b_bp = 'zeros'
        self.initializer_w_ap = 'vs'
        self.initializer_w_bp = 'vs'
        self.logdir = './results'
        self.lr = 0.1
        self.lr_decay_type = 'piecewise'
        self.optimizer = 'momentum'
        self.path_assess = self.logdir + '/assess'
        self.path_data = '/content/gdrive/MyDrive/Mini-Project'
        self.path_model = self.logdir + '/model'
        self.path_summary = self.logdir + '/summary'
        self.save_interval = 100
        self.target_sparsity = 0.9
        self.train_iterations = 30000
        self.is_sample = False
        self.sample_class = 0

# Comparisons under Varied Sparsity

In [ ]:
import os
import sys
import argparse
import tensorflow as tf

from dataset import Dataset
from model import Model
import prune
import train
import test


# Set arguments
args = Args()
args.arch = 'lenet300' # lenet300 or lenet5
args.batch_size = 500
args.datasource = 'mnist' # mnist or kmnist
args.decay_boundaries = [5000, 10000, 15000, 20000, 25000]
if args.arch == 'lenet300':
    args.decay_values = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
elif args.arch == 'lenet5':
    args.decay_values = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7]
args.target_sparsity = 0
args.train_iterations = 30000


for iter in range(0, 10):
    print("|--------- Iteration {} ---------|".format(iter))
    
    # Set arguments
    args.logdir = './VariedSparsity/iter' + str(iter)
    args.path_assess = args.logdir + '/assess'
    args.path_model = args.logdir + '/model'
    args.path_summary = args.logdir + '/summary'

    # Dataset
    dataset = Dataset(**vars(args))

    # Reset the default graph and set a graph-level seed
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    tf.reset_default_graph()
    tf.set_random_seed(iter)

    # Model
    model = Model(num_classes=dataset.num_classes, **vars(args))
    model.construct_model()

    # Session
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()

    # Prune
    prune.prune(args, model, sess, dataset)

    # Train and test
    train.train(args, model, sess, dataset)
    test.test(args, model, sess, dataset)

    sess.close()

# Comparisons with Multiple Network Architectures

In [ ]:
import os
import sys
import argparse
import tensorflow as tf

from dataset import Dataset
from model import Model
import prune
import train
import test


# Set arguments
args = Args()
# Set a network architecture
# Convolutional: alexnet-v1, alexnet-v2, vgg-c, vgg-d, vgg-like
# Residual: resnet-18, resnet-34
# https://openaccess.thecvf.com/content_cvpr_2016/papers/He_Deep_Residual_Learning_CVPR_2016_paper.pdf
# Squeeze: squeezenet-vanilla, squeezenet-bypass
# https://arxiv.org/pdf/1602.07360.pdf?ref=https://githubhelp.com
# Inception: googlenet
# https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Szegedy_Going_Deeper_With_2015_CVPR_paper.pdf
# Dense: densenet-121, densenet-169, densenet-201, densenet-264
# https://openaccess.thecvf.com/content_cvpr_2017/papers/Huang_Densely_Connected_Convolutional_CVPR_2017_paper.pdf
args.arch = 'resnet-18'
args.batch_size = 512
args.datasource = 'cifar-10'
args.decay_boundaries = [7500, 15000, 22500, 30000]
args.decay_values = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
args.target_sparsity = 0.95
args.train_iterations = 37500


for iter in range(0, 10):
    print("|--------- Iteration {} ---------|".format(iter))

    # Set arguments
    args.logdir = './NetworkArchitectures/iter' + str(iter)
    args.path_assess = args.logdir + '/assess'
    args.path_model = args.logdir + '/model'
    args.path_summary = args.logdir + '/summary'

    # Dataset
    dataset = Dataset(**vars(args))

    # Reset the default graph and set a graph-level seed
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    tf.reset_default_graph()
    tf.set_random_seed(iter)

    # Model
    model = Model(num_classes=dataset.num_classes, **vars(args))
    model.construct_model()

    # Session
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()

    # Prune
    prune.prune(args, model, sess, dataset)

    # Train and test
    train.train(args, model, sess, dataset)
    test.test(args, model, sess, dataset)

    sess.close()

# Understanding Which Connections Are Pruned

## The Visualizations

In [ ]:
import os
import sys
import argparse
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt

from dataset import Dataset
from model import Model

# Set arguments
args = Args()
args.arch = 'lenet300'
args.datasource = 'mnist' # mnist or kmnist


images = np.zeros((9, 10, 28, 28))


for sample_class in range(0, 10):
    print("|--------- Sample class {} ---------|".format(sample_class))
    for target_sparsity in range(1, 10):
        print("|--------- Target sparsity {} ---------|".format(target_sparsity * 0.1))

        # Set arguments
        args.is_sample = True
        args.sample_class = sample_class
        args.target_sparsity = target_sparsity * 0.1

        # Dataset
        sampleset = Dataset(**vars(args))

        # Reset the default graph and set a graph-level seed
        tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
        tf.reset_default_graph()
        tf.set_random_seed(target_sparsity)

        # Model
        model = Model(num_classes=dataset.num_classes, **vars(args))
        model.construct_model()

        # Session
        sess = tf.InteractiveSession()
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()

        # Prune
        print('|========= START PRUNING =========|')
        t_start = time.time()
        batch = sampleset.get_next_batch('train', 100)
        feed_dict = {}
        feed_dict.update({model.inputs[key]: batch[key] for key in ['input', 'label']})
        feed_dict.update({model.compress: True, model.is_train: False, model.pruned: False})
        result = sess.run([model.outputs, model.sparsity, model.mask['w1']], feed_dict)
        print('Pruning: {:.3f} global sparsity (t:{:.1f})'.format(result[1], time.time() - t_start))

        # Average and reshape the indicator matrix c_1
        w1_mask = np.array(result[-1])
        images[target_sparsity-1, sample_class] = np.reshape(np.mean(w1_mask, axis=1), (28, 28))

        sess.close()

In [ ]:
# Plot the visualizations of pruned parameters
fig, ax = plt.subplots(nrows=9, ncols=10, figsize=[28, 28], squeeze=True)
fig = plt.subplots_adjust(wspace=0.1, hspace=0.01)

for i, axes in enumerate(ax.flat):
    ir = i // 10
    ic = i % 10
    axes.imshow(images[ir, ic] * 255, cmap='gray', vmin=0, vmax=255)
    axes.axis('off')

## The Ratios

In [ ]:
import os
import sys
import argparse
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt

from dataset import Dataset
from model import Model


# Set arguments
args = Args()
args.arch = 'lenet300'
args.datasource = 'kmnist' # mnist or kmnist


ratios = np.zeros((10, 6))


for target_sparsity in range(0, 10):
    print("|--------- Target sparsity {} ---------|".format(target_sparsity * 0.1))

    # Set arguments
    args.target_sparsity = target_sparsity * 0.1

    # Dataset
    sampleset = Dataset(**vars(args))

    # Reset the default graph and set a graph-level seed
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    tf.reset_default_graph()
    tf.set_random_seed(target_sparsity)

    # Model
    model = Model(num_classes=dataset.num_classes, **vars(args))
    model.construct_model()

    # Session
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()

    # Prune
    print('|========= START PRUNING =========|')
    t_start = time.time()
    batch = sampleset.get_next_batch('train', 100)
    feed_dict = {}
    feed_dict.update({model.inputs[key]: batch[key] for key in ['input', 'label']})
    feed_dict.update({model.compress: True, model.is_train: False, model.pruned: False})
    result = sess.run([model.outputs, model.sparsity, model.mask], feed_dict)
    print('Pruning: {:.3f} global sparsity (t:{:.1f})'.format(result[1], time.time() - t_start))

    # Compute the ratios
    masks = result[-1]
    count = 0
    for key in masks.keys():
        if 'w' in key:
            mask = np.array(masks[key])
            ratios[target_sparsity, count] = np.count_nonzero(mask) / np.size(mask)
            ratios[target_sparsity, count+1] = 1 - ratios[target_sparsity, count]
            count += 2

    sess.close()

In [ ]:
for index in range(0, 10):
    print('{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f};'.format(
        ratios[index, 1], ratios[index, 0], ratios[index, 3], 
        ratios[index, 2], ratios[index, 5], ratios[index, 4]))

# Effects of Batch Sizes

## Batch Size for Pruning

In [ ]:
import os
import sys
import argparse
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt

from dataset import Dataset
from model import Model
import train
import test


# Set arguments
args = Args()
args.arch = 'lenet300' # lenet300 or lenet5
args.batch_size = 500
args.datasource = 'mnist' # mnist or kmnist
args.decay_boundaries = [5000, 10000, 15000, 20000, 25000]
args.decay_values = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
args.target_sparsity = 0.9
args.train_iterations = 30000


Dbs = [1, 10, 100, 1000, 10000, 54000]
images = np.zeros((6, 28, 28))


for iter in range(0, 10):
    print("|--------- Iteration {} ---------|".format(iter))
    for Db in Dbs:
        print("|--------- |D^b| {} ---------|".format(Db))

        # Set arguments
        args.logdir = './BatchSizes1/iter' + str(iter) + 'Db' + str(Db)
        args.path_assess = args.logdir + '/assess'
        args.path_model = args.logdir + '/model'
        args.path_summary = args.logdir + '/summary'

        # Dataset
        dataset = Dataset(**vars(args))

        # Reset the default graph and set a graph-level seed
        tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
        tf.reset_default_graph()
        tf.set_random_seed(iter)

        # Model
        model = Model(num_classes=dataset.num_classes, **vars(args))
        model.construct_model()

        # Session
        sess = tf.InteractiveSession()
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()

        # Prune
        print('|========= START PRUNING =========|')
        t_start = time.time()
        batch = dataset.get_next_batch('train', Db)
        feed_dict = {}
        feed_dict.update({model.inputs[key]: batch[key] for key in ['input', 'label']})
        feed_dict.update({model.compress: True, model.is_train: False, model.pruned: False})
        result = sess.run([model.outputs, model.sparsity, model.mask['w1']], feed_dict)
        print('Pruning: {:.3f} global sparsity (t:{:.1f})'.format(result[1], time.time() - t_start))

        # Average and reshape the indicator matrix c_1
        w1_mask = np.array(result[-1])
        images[Dbs.index(Db)] = np.reshape(np.mean(w1_mask, axis=1), (28, 28))

        # Train and test
        train.train(args, model, sess, dataset)
        test.test(args, model, sess, dataset)

        sess.close()

In [ ]:
# Plot and download the visualizations of pruned parameters
from google.colab import files

for i in range(0, 6):
    fig_name = args.datasource + str(Dbs[i]) + '.png'
    fig = plt.imshow(images[i] * 255, cmap='gray')
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig(fig_name, bbox_inches='tight', pad_inches = 0)
    files.download(fig_name) 

## Batch Sizes for Pruning and Training

In [ ]:
import os
import sys
import argparse
import tensorflow as tf

from dataset import Dataset
from model import Model
import prune
import train
import test


# Set arguments
args = Args()
args.arch = 'lenet300'
args.datasource = 'mnist' # mnist or kmnist
args.decay_boundaries = [5000, 10000, 15000, 20000, 25000]
args.decay_values = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
args.target_sparsity = 0.9 # 0.1, 0.5, 0.9
args.train_iterations = 30000


batch_sizes = [50, 100, 500, 1000]


for batch_size in batch_sizes:
    train_iterations = int(150000 / (batch_size / 100))
    decay_boundaries = []
    for time in range(1, 6):
        decay_boundaries.append(int(train_iterations / 6 * time))

    for iter in range(0, 5):
        print("|--------- Iteration {} ---------|".format(iter))

        # Set arguments
        args.batch_size = batch_size
        args.logdir = './BatchSizes2/batch' + str(batch_size) + 'iter' + str(iter)
        args.path_assess = args.logdir + '/assess'
        args.path_model = args.logdir + '/model'
        args.path_summary = args.logdir + '/summary'

        # Dataset
        dataset = Dataset(**vars(args))

        # Reset the default graph and set a graph-level seed
        tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
        tf.reset_default_graph()
        tf.set_random_seed(iter)

        # Model
        model = Model(num_classes=dataset.num_classes, **vars(args))
        model.construct_model()

        # Session
        sess = tf.InteractiveSession()
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()

        # Prune
        prune.prune(args, model, sess, dataset)

        # Train and test
        train.train(args, model, sess, dataset)
        test.test(args, model, sess, dataset)

        sess.close()